# Statystyka matematyczna - ćwiczenia laboratoryjne 2023/2024

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (nazwa zespołu oraz imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

Nazwa zespołu:
Członkowie:

---

# Zestaw zadań 10: Weryfikacja hipotez statystycznych - testy nieparametryczne

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów

### Funkcje do wykorzystania

In [3]:
def vizualize_test_result(pdf, statistic, rejection_region):
    """
    Parameters
    -------
    pdf: pandas.DataFrame
        Tablica zawierająca informacje niezbędne do wykreślenia funkcji gęstości prawdopodobieństwa wykorzystywanego w teście rozkładu
        teoretycznego - index z wariantami zmiennej losowej i nadaną nazwą zmiennej (np. x) oraz pojedynczą kolumnę z wartościami
        gęstościami prawdopodobieństwa wyliczonymi dla poszczególnych wariantów o nadanej nazwie bazującej na nazwie zmiennej (np. f(x)).
    statistic: float
        Wartość statystyki testowej wyliczonej na podstawie próby losowej.
    rejection_region: list
        Lista zawierająca trzy elementy pozwalające na określenie obszaru krytycznego w następującej kolejności:
         - wartość dolnej granicy przedziału,
         - wartość górnej granicy przedziału,
         - "inner" lub "outer" w zależności od tego, czy ma być zakolorowana wewnętrzna, czy zewnętrzna część przedziału.
    """
    pdf_name= pdf.columns[0]
    
    fig, axes = plt.subplots(1, 1, figsize=(8, 3), facecolor='white')

    axes.plot(pdf.index, pdf[pdf_name], color="grey")

    if rejection_region[2]=="inner":
        axes.fill_between(pdf.index, pdf[pdf_name], 0, where=((pdf.index > rejection_region[0]) & (pdf.index<rejection_region[1])), color='red', alpha=0.5)
    elif rejection_region[2]=="outer":
        axes.fill_between(pdf.index, pdf[pdf_name], 0, where=((pdf.index < rejection_region[0]) | (pdf.index>rejection_region[1])), color='red', alpha=0.5)
    
    axes.vlines(x=statistic, ymin=0, ymax=np.max(pdf[pdf_name]/3), color="blue")
    
    axes.set_xlabel(pdf.index.name)
    axes.set_ylabel(pdf_name)

### Zadanie 1: Test zgodności chi-kwadrat [10 pkt]

Zmienna `sample` zawiera wyniki 50 krotnego powtórzenia doświadczenia, w którym rzucamy monetą 20 razy i zapisujemy liczbę orłów, które wypadły w serii rzutów.

Przygotuj dane oraz napisz funkcję `chi2_homogeneity_NHST()`, która zostanie wykorzystana do weryfikacji hipotezy, że wykorzystana w doswiadczeniu moneta była uczciwa (prawdopodobieństwo wyrzucenia orła i reszki jest równe).

In [4]:
sample = np.array([ 9, 10, 11, 11,  8,  7, 14,  7, 14, 12,
                   14, 11,  9, 11, 15, 14, 11, 12,  8, 13,
                    9, 12,  7,  8, 10,  9, 14,  9,  8, 11,
                    8, 10,  5, 10,  8, 10, 13, 10, 14, 10,
                    8,  9, 14, 14,  8, 10,  9, 13, 10, 14])

#### a) Przygotowanie danych

Oblicz prawdopodobieństwa wystąpienia określonej liczby sukcesów (0, 1, 2 ...) w 20 niezależnych powtórzeniach doświaczenia o prawdopodobieństwie sukcesu równym 0.5. Wyniki zestaw w tabeli `theoretical_distribution` o następującej strukturze:

> Indeks:</br>
> x - warianty przyjmowane przez zmienną X.</br>

> Kolumny:</br>
> P(x) - prawdopodobieństwo wystąpienia poszczególnych wariantów.

Przygotuj szereg rozdzielczy wystąpienia określonej liczby orłów w seriach 20 rzutów monetą. Wyniki zestaw w tabeli `tested_distribution` o następującej strukturze:

> Indeks:</br>
> x - warianty przyjmowane przez zmienną X (identyczne jak w poprzedniej tablicy).</br>

> Kolumny:</br>
> N(x) - zliczenia wystąpienia poszczególnych wariantów.

In [5]:
# Tabela funkcji rozkładu prawdopodobieństwa rozkładu teoretycznego
# YOUR CODE HERE




n = 20 
p = 0.5  
x = range(n + 1) 
probabilities =[0] * (n+1)
for i in range(len(x)):
    probabilities[i] = sp.stats.binom.pmf(i, n, p)

#probabilities = [sp.stats.binom.pmf(i, n, p) for i in x]  
theoretical_distribution = pd.DataFrame(data = probabilities, index = x, columns = ["P(x)"])
theoretical_distribution.index.name = 'x'

print(theoretical_distribution)



# moja wersja 
#x = np.linspace(0,20,21)
#print(x)
#p = 0.5
#P_X = sp.stats.binom.pmf(x,20,p)

#theoretical_distribution = pd.DataFrame(data = P_X, index = x, columns = ["P(x)"])
#theoretical_distribution.index.name = 'x'

#print(theoretical_distribution)
      

            P(x)
x               
0   9.536743e-07
1   1.907349e-05
2   1.811981e-04
3   1.087189e-03
4   4.620552e-03
5   1.478577e-02
6   3.696442e-02
7   7.392883e-02
8   1.201344e-01
9   1.601791e-01
10  1.761971e-01
11  1.601791e-01
12  1.201344e-01
13  7.392883e-02
14  3.696442e-02
15  1.478577e-02
16  4.620552e-03
17  1.087189e-03
18  1.811981e-04
19  1.907349e-05
20  9.536743e-07


In [6]:
# Komórka testowa
assert type(theoretical_distribution) == pd.DataFrame
assert theoretical_distribution.shape == (21, 1)
assert list(theoretical_distribution.columns) == ["P(x)"]
assert theoretical_distribution.index.name == 'x'
assert np.isclose(np.sum(theoretical_distribution.index.values), 210)
assert np.isclose(np.sum(theoretical_distribution["P(x)"]), 1)

In [30]:
# Tabela szeregu rozdzielczego próby losowej
# YOUR CODE HERE
x = theoretical_distribution.index.values
counts =[0] * (n+1)
for i in range(len(x)):
    counts[i] = np.sum(sample == i)
    
#counts = [np.sum(sample == i) for i in x]

tested_distribution = pd.DataFrame(data = counts,  index=x, columns=["N(x)"])
tested_distribution.index.name = 'x'

print(tested_distribution)


# moja wersja 

#N_X = [sp.stats.binom.pmf(i, 20, p) * 50 for i in x] # dajemy 50 probek 
#tested_distribution = pd.DataFrame(data = N_X, index = x, columns = ["N(x)"])
#tested_distribution.index.name = 'x'
#print(tested_distribution)

    N(x)
x       
0      0
1      0
2      0
3      0
4      0
5      1
6      0
7      3
8      8
9      7
10     9
11     6
12     3
13     3
14     9
15     1
16     0
17     0
18     0
19     0
20     0


In [31]:
# Komórka testowa
assert type(tested_distribution) == pd.DataFrame
assert tested_distribution.shape == (21, 1)
assert list(tested_distribution.columns) == ["N(x)"]
assert tested_distribution.index.name == 'x'
assert np.isclose(np.sum(tested_distribution.index.values), 210)
assert np.isclose(np.sum(tested_distribution["N(x)"]), 50)

#### b) Test zgodności chi-kwadrat

Przygotuj funkcję `chi2_NHST()`, która będzie weryfikować test zgodności chi-kwadrat i wyświetlać wynik testu statystycznego zgodnie z wytycznymi zawartymi w dołączonym do notatnika zestawie wzorów, w treści zadania oraz w docstring funkcji.

Poza zwracaniem zmiennych wymienionych w docsting funkcji, które są potrzebne do przeprowadzenia automatycznych testów funkcji powinna wykorzystywać udostępnioną na początku notatnika funkcję `vizualize_test_result()` do generowania wykresu zawierającego:
- funkcję gęstości prawdopodobieństwa rozkładu teoretycznego wykorzystywanego w teście w zakresie od wartości, dla której dystrybuanta tego rozkładu przyjmuje wartość 0.0001 do wartości, dla której dystrybuanta tego rozkładu przyjmuje wartość 0.9999,
- zaznaczony wyróżniającym się kolorem fragmentem pola pod krzywą odpowiadający wyznaczonemu w teście obszarowi krytycznemu,
- wyróżniającą się kolorem pionowę linię wskazującą obliczoną wartość statystyki.

In [7]:
def chi2_NHST(tested_distribution, theoretical_distribution, alpha):
    """
    Parameters
    -------
    tested_distribution: DataFrame
        Tablica zawierająca indeks o nazwie "x" z wariantami zmiennej w próbie losowej oraz kolumnę "N(x)" z zliczeniami wariantów.
    theoretical_distribution: DataFrame
        Tablica zawierająca indeks o nazwie "x" z wariantami zmiennej teoretycznej oraz kolumnę "P(x)" z prawdopodobieństwami wariantów.
    alpha: float
        Wartość określająca poziom istotności.
    
    Returns
    -------
    chi2: float
        Wyliczona na podstawie próby losowej wartość statystyki chi2.
    chi2_alpha: float
        Wartość statystyki chi2_alpha.
    H: int
        Wynik testu statystycznego, przyjmuje wartość:
        0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha,
        1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha.
    """
    # YOUR CODE HERE
    ni = tested_distribution['N(x)'].values
    print(ni)
    pi = theoretical_distribution['P(x)'].values
    print(pi)
    n = ni.sum()
    print(n)
    res = n * pi
    df=len(ni) - 1
    
    chi2 = ((ni - res)**2 / res).sum()
    chi2_alpha = sp.stats.chi2.ppf(1 - alpha, df)
    
    H = int(chi2 >= chi2_alpha)

    x = np.linspace(sp.stats.chi2.ppf(0.0001, df), sp.stats.chi2.ppf(0.9999, df), 1000)
    y = sp.stats.chi2.pdf(x, df)
    pdf = pd.DataFrame(data=y, index=x, columns=['pdf'])
    
    # Obszar krytyczny
    rejection_region = [chi2_alpha, np.inf, "inner"]
    
    # Wizualizacja
    vizualize_test_result(pdf, chi2, rejection_region)
    plt.title(f'Wizualizacja wyników przy założeniu poziomu ufności 1-alpha={1-alpha}')
    
    return chi2, chi2_alpha, H

In [8]:
# Komórka testowa
assert np.all(np.isclose(chi2_NHST(tested_distribution, theoretical_distribution, 0.02), (33.3612131026868, 35.01962554059928, 0)))
assert np.all(np.isclose(chi2_NHST(tested_distribution, theoretical_distribution, 0.05), (33.3612131026868, 31.410432844230918, 1)))

NameError: name 'tested_distribution' is not defined

### Zadanie 2: Weryfikacja hipotez z wykorzystaniem dedykowanych funkcji [5-10 pkt]

Zmienna `penguins` zawiera wyniki badań przeprowadzonych na pingwinach trzech różnych gatunków.

In [39]:
penguins = sns.load_dataset("penguins")
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


#### a) 

Korzystając z funkcji `scipy.stats.shapiro()` zweryfikuj hipotezę, że głębokość dzioba pingwinów gatunku Gentoo ma rozkład normalny. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.95.

Zapisz obliczoną wartość pvalue do zmiennej `pvalue_1`, oraz przygotuj zmienną `H_1` zawierającą wynik testu statystycznego (0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha, 1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha).

Uwagi do wykonania zadania:
 - ustaw następującą wartość argumentu funkcji wykonującej test statystyczny odpowiedzialnego za radzenie sobie z wartościami nan: `nan_policy="omit"`.

In [41]:
# YOUR CODE HERE
mask = penguins["species"] == "Gentoo"
print(mask)
temp = penguins[mask]
print(temp)
pvalue_1 = sp.stats.shapiro(temp["bill_depth_mm"].dropna()).pvalue

alpha = 0.05
H_1 = 0
if pvalue_1 < alpha:
    H_1 = 1

print(H_1)
print(pvalue_1)

0      False
1      False
2      False
3      False
4      False
       ...  
339     True
340     True
341     True
342     True
343     True
Name: species, Length: 344, dtype: bool
    species  island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
220  Gentoo  Biscoe            46.1           13.2              211.0   
221  Gentoo  Biscoe            50.0           16.3              230.0   
222  Gentoo  Biscoe            48.7           14.1              210.0   
223  Gentoo  Biscoe            50.0           15.2              218.0   
224  Gentoo  Biscoe            47.6           14.5              215.0   
..      ...     ...             ...            ...                ...   
339  Gentoo  Biscoe             NaN            NaN                NaN   
340  Gentoo  Biscoe            46.8           14.3              215.0   
341  Gentoo  Biscoe            50.4           15.7              222.0   
342  Gentoo  Biscoe            45.2           14.8              212.0   
343  Gentoo  B

In [42]:
# Komórka testowa
assert np.isclose(pvalue_1, 0.027695094782335943)
assert H_1 == 1

#### b) 

Korzystając z funkcji `scipy.stats.ks_1samp()` lub `scipy.stats.kstest()` zweryfikuj hipotezę, że długość skrzydeł pingwinów gatunku Adelie ma rozkład normalny o wartości oczekiwanej 190 i odchyleniu standardowym 6. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Zapisz obliczoną wartość pvalue do zmiennej `pvalue_2`, oraz przygotuj zmienną `H_2` zawierającą wynik testu statystycznego (0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha, 1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha).

Uwagi do wykonania zadania:
 - ustaw następującą wartość argumentu funkcji wykonującej test statystyczny odpowiedzialnego za radzenie sobie z wartościami nan: `nan_policy="omit"`.

In [15]:
mask = penguins["species"] == "Adelie"
temp = penguins[mask]
wing_length = temp["flipper_length_mm"].dropna()
print(wing_length)
x = sp.stats.norm(loc = 190, scale = 6)
ks_stat, pvalue_2 = sp.stats.kstest(wing_length, x.cdf, nan_policy= "omit")
H_2 = 0
alpha = 0.1
if pvalue_2 < alpha:
    H_2 = 1

print(H_2)
print(pvalue_2)
print(f"Wartość pvalue wynosi: {pvalue_2}")


0      181.0
1      186.0
2      195.0
4      193.0
5      190.0
       ...  
147    184.0
148    195.0
149    193.0
150    187.0
151    201.0
Name: flipper_length_mm, Length: 151, dtype: float64
0
0.4388010272246816
Wartość pvalue wynosi: 0.4388010272246816


In [16]:
# Komórka testowa
assert np.isclose(pvalue_2, 0.4388010272246816)
assert H_2 == 0

#### c) 

Korzystając z funkcji `scipy.stats.chisquare()` zweryfikuj hipotezę, że długość skrzydeł pingwinów gatunku Adelie ma rozkład normalny o wartości oczekiwanej 190 i odchyleniu standardowym 6. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.9.

Zapisz obliczoną wartość pvalue do zmiennej `pvalue_3`, oraz przygotuj zmienną `H_3` zawierającą wynik testu statystycznego (0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha, 1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha).

Schemat wykonania zadania:

Uwagi do wykonanai zadania:
 - korzystając z funkcji `numpy.arange()` lub `numpy.linspace()` przygotuj macierz z granicami następujących przedziałów: [170, 173), [173, 176), ... , [209, 212],
 - korzystając z fukcji `numpy.histogram()` podziel zakres danych i zlicz ile wartości badanej cechy znajduje się w każdym z przedziałów,
 - korzystając z funkcji modułu `scipy.stats.norm` oblicz jakie jest prawdopodobieństwo wylosowania wartości z poszczególnych przedziałów dla testowanego rozkładu normalnego,
 - przelicz prawdopodobieństwa na liczebności toretyczne (ustandaryzuj je do sumy równej liczebności testowanego zestawu danych),
 - wykonaj test statystyczny korzysztając z funkcji `scipy.stats.chisquare()`.

In [17]:
flipper_lengths = penguins[penguins['species'] == 'Adelie']['flipper_length_mm'].dropna().values

mu = 190
sigma = 6

bins = np.arange(170, 213, 3)
observed_counts, bin_edges = np.histogram(flipper_lengths, bins=bins)

# oblcizamy  teoretyczne liczebności
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
expected_probs = sp.stats.norm.cdf(bin_edges[1:], mu, sigma) - sp.stats.norm.cdf(bin_edges[:-1], mu, sigma)
expected_counts = expected_probs * len(flipper_lengths)

expected_counts *= observed_counts.sum() / expected_counts.sum()

#  test chi-kwadrat
chi2_stat, pvalue_3 = sp.stats.chisquare(observed_counts, expected_counts)

# weryfikacja hipotezy
alpha = 0.1
H_3 = 1 if pvalue_3 < alpha else 0
pvalue_3

0.4216647220715652

In [18]:
# Komórka testowa
assert np.isclose(pvalue_3, 0.4216647220715661)
assert H_3 == 0

### d)

Korzystając z funkcji `scipy.stats.ks_2samp()` lub `scipy.stats.kstest()` zweryfikuj hipotezę, że długość skrzydeł pingwinów gatunku Adelie i Chinstrap mają taki sam rozkład statystyczny. W obliczeniach przyjmij poziom istotności 1 - alpha = 0.98.

Zapisz obliczoną wartość pvalue do zmiennej `pvalue_4`, oraz przygotuj zmienną `H_4` zawierającą wynik testu statystycznego (0 - gdy wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności 1-alpha, 1 - gdy następuje odrzucenie H0 na rzecz H1 na poziomie istotności 1-alpha).

Uwagi do wykonania zadania:
 - ustaw następującą wartość argumentu funkcji wykonującej test statystyczny odpowiedzialnego za radzenie sobie z wartościami nan: `nan_policy="omit"`.

In [45]:
mask1 = penguins["species"] == "Adelie"
mask2 = penguins["species"] == "Chinstrap"
temp1 = penguins[mask1]
temp2 = penguins[mask2]
print(temp2)
wing_length1 = temp1["flipper_length_mm"].dropna()
wing_length2 = temp2["flipper_length_mm"].dropna()
print(wing_length2)
ks_stat, pvalue_4 = sp.stats.ks_2samp(wing_length1, wing_length2,nan_policy='omit')
H_4 = 0
alpha = 0.02
if pvalue_4 < alpha:
    H_4 = 1
print(H_4)
print(pvalue_4)

                                    

       species island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
152  Chinstrap  Dream            46.5           17.9              192.0   
153  Chinstrap  Dream            50.0           19.5              196.0   
154  Chinstrap  Dream            51.3           19.2              193.0   
155  Chinstrap  Dream            45.4           18.7              188.0   
156  Chinstrap  Dream            52.7           19.8              197.0   
..         ...    ...             ...            ...                ...   
215  Chinstrap  Dream            55.8           19.8              207.0   
216  Chinstrap  Dream            43.5           18.1              202.0   
217  Chinstrap  Dream            49.6           18.2              193.0   
218  Chinstrap  Dream            50.8           19.0              210.0   
219  Chinstrap  Dream            50.2           18.7              198.0   

     body_mass_g     sex  
152       3500.0  Female  
153       3900.0    Male  
154       3650.0  

In [44]:
assert np.isclose(pvalue_4, 5.6295807497561385e-06)
assert H_4 == 1